# Laboratoire 2 : Arbre de désision, Bayes naïf et KNN
#### Département du génie logiciel et des technologies de l’information

| Étudiants             |                                                         |
|-----------------------|---------------------------------------------------------|
| Jean-Philippe Decoste |  DECJ19059105                                           |
| Ahmad Al-Taher        |   ALTA22109307                                          |
| Cours                 | GTI770 - Systèmes intelligents et apprentissage machine |
| Session               | Automne 2018                                            |
| Groupe                | 2                                                       |
| Numéro du laboratoire | 02                                                      |
| Professeur            | Hervé Lombaert                                          |
| Chargé de laboratoire | Pierre-Luc Delisle                                      |
| Date                  | 30 oct 2018                                             |

In [41]:
import math

import cv2 as cv
import graphviz
import matplotlib.pyplot as plt
import numpy as np
from scipy.misc import face, imread, imshow
from skimage import img_as_ubyte
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from sklearn import preprocessing, tree
from sklearn.datasets import load_iris

In [109]:
class Email:    
    def __init__(self, features):
        self.features = features
        self.spam = int(self.features[len(self.features)-1]) == 1
        del self.features[-1]


In [110]:
class Image:    
    def __init__(self, features):
        self.features = features
        self.spiral = self.features[len(self.features)-1] == 1
        del self.features[-1]
        self.id = int(self.features[0]) 
        del self.features[0]

In [113]:
def loadEmailFeatures(csvPath):
    """
    This method is used to al the email features from the data set.
    
    Args:
        csvPath: the dataset file path
    """
    csvFile = open(csvPath, 'r') # option r veut dire read
    #index is used to limit the numbero of data for test purposes
    index = 0
    for line in csvFile:
        features = line.split(",")
        if len(features) > 0:
            Emails.append(Email(features))
        
        index +=1
        if index == 100:
            break

    csvFile.close()

def loadImageFeatures(csvPath):
    """
    This method is used to al the email features from the data set.
    
    Args:
        csvPath: the dataset file path
    """
    csvFile = open(csvPath, 'r') # option r veut dire read
    #index is used to limit the numbero of data for test purposes
    index = 0
    for line in csvFile:
        features = line.split(",")
        if len(features) > 0:            
            cleanCsvFeatures(features)
            Images.append(Image(features))
        index +=1
        if index == 100:
            break

    csvFile.close()

def cleanCsvFeatures(features):
    #clean up the features
    #the csv files how the data as x.xxxxe+XX and we want tem as x.xxx
    for i,val in enumerate(features):
        features[i] = float(val)


In [116]:
Emails = []
Images = []
loadEmailFeatures(r"C:\Users\ThinkPad\Downloads\gti770\data\csv\spam\spam.csv")
loadImageFeatures(r"C:\Users\ThinkPad\Downloads\gti770\data\csv\galaxy\galaxy_feature_vectors.csv")

print(Emails[0].features)
print(Emails[0].spam)
print(Images[0].features)
print(Images[0].spiral)

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0.43', '0', '0.28', '0', '0', '0', '1.43', '1.15', '0.86', '0.28', '0.28', '0.28', '0.28', '0.28', '0.14', '0.28', '0.28', '0.28', '0.28', '0', '0.14', '0.28', '0', '0', '0.43', '0', '0.57', '0.28', '0', '0', '0.023', '0.324', '0.046', '0', '0', '0', '2.24', '12', '372']
False
[43.930924087036736, 51.01459655976993, 57.710347821368494, 10.04080181393207, 11.753778539411638, 12.423907531431968, 20.01964231316385, 20.145107092058197, 23.890885542842724, 18.698736310974507, 15.543446679530906, 17.429573092368198, 3.860164803435622, 3.685242829417392, 3.8499784988736665, 0.35608710094600127, 0.5968399839318662, 0.5457711835289252, 0.5062480342359442, 171.400450796202, 86.77114128273193, 2849625.0, 7.1205854415893555, 0.30214676507437943, 243.95547057235152, 0.0841936632253471, 3.403742192024813, 0.2241284442662666, 0.9342236609188999, 1.0, -0.26520337421586726, 0.9519712600246981, 1.159898581793148,

## Introduction

## Question 1
### Méthode de création des ensembles de données

## Question 1
### Détails des ensembles produits

## Question 2
### Approche de validation proposée et justification

## Question 3
### Matrice des expérimentations

## Question 3
### Étude des hyperparamètres et des modèles

## Question 4
### Impact de la taille des ensembles de données sur la performance de classification

## Question 5
### Impact du bruit dans les ensembles de données sur la performance de classification

## Question 6
### Discussion sur la nature des données

## Question 7
### Recommandations

## Question 8
### Améliorations possibles

## Conclusion

## Bibliographie